In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim # torch  선언
torch.manual_seed(1)

import random
random.seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
      torch.cuda.manual_seed_all(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option('mode.chained_assignment',  None)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sejongai19011494/train-1.csv
/kaggle/input/sejongai19011494/test-1.csv
/kaggle/input/sejongai19011494/submission.csv


In [2]:
X = pd.read_csv('/kaggle/input/sejongai19011494/train-1.csv')
y = X['건당이용금액']
test = pd.read_csv('/kaggle/input/sejongai19011494/test-1.csv')
X.head()

,기준년월,강수량,평균기온,최고기온,최저기온,제주 대분류,제주 중분류,업종명,성별,연령대별,카드이용건수,건당이용금액
0,2015-10,61.0,18.9,25.5,10.7,서귀포시,예래동,슈퍼 마켓,여,20대,30,10881
1,2015-09,172.9,23.2,28.6,18.4,제주시,연동,약국,여,30대,239,12597
2,2015-11,215.5,15.4,22.6,2.1,서귀포시,남원읍,슈퍼 마켓,여,40대,131,21265
3,2016-04,232.2,15.8,22.7,10.8,서귀포시,예래동,기념품 점,여,40대,29,17000
4,2016-04,121.4,15.6,24.5,9.8,제주시,연동,슈퍼 마켓,여,50대,369,22050


In [3]:
X['연도'] = X['기준년월'].str.split('-').str[0].astype(int)
X['월'] = X['기준년월'].str.split('-').str[1].astype(int)
X = X.drop([ '기준년월', '건당이용금액'], axis=1)

test['연도'] = test['기준년월'].str.split('-').str[0].astype(int)
test['월'] = test['기준년월'].str.split('-').str[1].astype(int)
test = test.drop(['기준년월'], axis=1)

In [4]:
list =['제주 대분류', '제주 중분류', '업종명', '성별', '연령대별']

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for c in list:
    label = pd.concat([X[c], test[c]], axis=0)
    le.fit(label)
    X[c] = le.transform(X[c])
    test[c] = le.transform(test[c])

In [5]:
X.describe()

,강수량,평균기온,최고기온,최저기온,제주 대분류,제주 중분류,업종명,성별,연령대별,카드이용건수,연도,월
count,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000,10516.000000
mean,144.535470,16.901883,25.847651,9.261316,0.662800,4.627615,6.634557,0.501712,1.485926,132.430202,2015.192659,6.492202
std,83.672119,6.972154,5.999427,8.190967,0.472776,2.860087,3.458367,0.500021,1.088081,210.386765,0.685401,3.386420
min,31.000000,5.000000,14.700000,-6.900000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,2014.000000,1.000000
25%,90.000000,10.400000,20.500000,1.100000,0.000000,2.000000,4.000000,0.000000,1.000000,23.000000,2015.000000,4.000000
50%,121.400000,18.800000,27.500000,10.800000,1.000000,5.000000,6.000000,1.000000,1.000000,54.000000,2015.000000,7.000000
75%,183.500000,22.800000,30.200000,16.800000,1.000000,7.000000,11.000000,1.000000,2.000000,142.250000,2016.000000,9.000000
max,426.500000,28.500000,36.700000,22.300000,1.000000,9.000000,13.000000,1.000000,3.000000,2574.000000,2016.000000,12.000000


In [6]:
q1, q3 = np.percentile(X['카드이용건수'], [25, 75])
iqr = q3 - q1
lower_bound = q1 - (iqr * 1.5)
upper_bound = q3 + (iqr * 1.5)
X['카드이용건수'][X['카드이용건수']<lower_bound] = lower_bound
X['카드이용건수'][X['카드이용건수']>upper_bound] = upper_bound
test['카드이용건수'][test['카드이용건수']<lower_bound] = lower_bound
test['카드이용건수'][test['카드이용건수']>upper_bound] = upper_bound


In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)
test_sc = sc.transform(test)

In [8]:
X = torch.FloatTensor(X.to_numpy()).to(device)
test = torch.FloatTensor(test.to_numpy()).to(device)
y = torch.FloatTensor(y.to_numpy()).to(device)

X_sc = torch.FloatTensor(X_sc).to(device)
test_sc = torch.FloatTensor(test_sc).to(device)

In [9]:
# DNN 모델 만들기
linear1 = torch.nn.Linear(X.shape[1], 512)
linear2 = torch.nn.Linear(512, 256)
linear3 = torch.nn.Linear(256, 128)
linear4 = torch.nn.Linear(128, 1)
relu = torch.nn.LeakyReLU() 
dropout = torch.nn.Dropout(0.2)

# weight 초기화
torch.nn.init.kaiming_normal_(linear1.weight)
torch.nn.init.kaiming_normal_(linear2.weight)
torch.nn.init.kaiming_normal_(linear3.weight)
torch.nn.init.kaiming_normal_(linear4.weight)

model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4).to(device)

In [10]:
# 모델 학습
model.train()
loss = torch.nn.L1Loss() 
optimizer = optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(2500):
    hypothesis = model(X_sc)
    cost = loss(hypothesis, y.unsqueeze(1))
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch%100==0:
        print(epoch, cost.item())

0 35100.1875
100 19547.732421875
200 18785.833984375
300 18123.76171875
400 17167.37890625
500 15810.4375
600 14180.951171875
700 12301.9306640625
800 11106.6875
900 10143.7333984375
1000 9418.3056640625
1100 9002.1611328125
1200 8629.2900390625
1300 8424.015625
1400 8269.205078125
1500 8086.28857421875
1600 8060.53369140625
1700 7905.39404296875
1800 7779.8330078125
1900 7869.3056640625
2000 7635.001953125
2100 7617.38037109375
2200 7458.2529296875
2300 7398.6103515625
2400 7330.29833984375


In [11]:
model.eval()
with torch.no_grad():
    predict = model(test_sc).cpu()
    predict = predict.detach().numpy().squeeze(1)


In [12]:
submission = pd.read_csv('/kaggle/input/sejongai19011494/submission.csv')
submission['Predicted'] = predict

submission.to_csv("submission.csv", index=False)
submission.head()

,Id,Predicted
0,0,16259.269531
1,1,12710.473633
2,2,12513.958984
3,3,35875.226562
4,4,13250.591797
